In [1]:
from rpy2.robjects.packages import importr
utils = importr("utils")
extremes = importr("extRemes")
ismev = importr("ismev")
eva = importr("eva")
gnFit = importr("gnFit")
from netCDF4 import Dataset
import xarray as xr
import matplotlib.pyplot as plt
import time
import numpy as np
import pandas as pd
from datetime import datetime
from matplotlib.offsetbox import AnchoredText
from netCDF4 import num2date, date2num
statis = importr('stats')
import rpy2.robjects.numpy2ri
rpy2.robjects.numpy2ri.activate()
import os
os.chdir(r"C:\Users\Pranjali\Downloads")

In [2]:
climate = xr.open_dataset("Climate.nc")
climate1 = climate['precipitation1'].sel(time = slice('1951-01-01','1980-12-31'))
climate2 = climate['precipitation1'].sel(time = slice('1981-01-01','2015-12-31'))

In [5]:
l=[climate1,climate2]
l1=[]

In [7]:
for j in l:
    ppt = j.where(j>=0)
    ppt2 = j.rolling(time=2).sum()
    maxima1 = ppt2.groupby('time.year').max('time')
    rlv2=[]
    rlv302=[]
    paramlist2=[]
    lon1=[]
    lat1=[]
    for i in range(129):
        for j in range(135):

            maxima11 = maxima1.sel(lat=climate.lat[i],lon=climate.lon[j])
            params1 = {}    
            if(maxima11.values[1]>0):
            #RL2 = []
                block_maxima=maxima11.values[~np.isnan(maxima11.values)]
                try:
                    gev_fit =  eva.gevrFit(block_maxima)
                except:
                    pass
                MLE_est = list(gev_fit.rx2('par.ests'))

            #print(block_maxima)


                params1['location'] = MLE_est[0]

                params1['scale'] =  MLE_est[1] 

                params1['shape'] = MLE_est[2]

                if MLE_est[1]<0:
                    MLE_est[1] = 0.01

                y = eva.rgevr(10000, 1, loc = MLE_est[0], scale = MLE_est[1], shape = MLE_est[2])

                sig_test = statis.ks_test(block_maxima, y)

                params1['p_val'] = sig_test.rx2('p.value')[0]
                paramlist2.append(params1)

    #print(params)
  


                if params1['p_val']>0.05:

                    temp = eva.gevrRl(gev_fit, 100, method = "delta")
                    temp1 = eva.gevrRl(gev_fit, 30, method = "delta")
                    RL1 = temp.rx2('Estimate')[0]
                    RL301 = temp1.rx2('Estimate')[0]
#n['RLV2'][i][j] = RL2

                else:
                    params1['p_val'] = np.nan
                    params1['location'] = np.nan
                    params1['scale'] = np.nan
                    params1['shape'] = np.nan
            #print(RL2,climate.lat.values[i],climate.lon.values[j])
                rlv2.append(RL1)
                rlv302.append(RL301)
                lat1.append(climate.lat.values[i])
                lon1.append(climate.lon.values[j])
#n['RLV2'][i][j] = np.nan
#print(RL2)
            else:
                params1['p_val'] = np.nan
                params1['location'] = np.nan
                params1['scale'] = np.nan
                params1['shape'] = np.nan
    df1 = pd.DataFrame({'rlv2':rlv2,'rlv302':rlv302,'lat1':lat1,'lon1':lon1})
    l1.append(df1)

In [9]:
df0=l1[0]
df1=l1[1]
ds0= df0.set_index(['lat1','lon1']).to_xarray()
ds0.to_netcdf(path=r'D:\ds2p1951-1880-30100.nc')
ds1= df1.set_index(['lat1','lon1']).to_xarray()
ds1.to_netcdf(path=r'D:\ds2p1981-2015-30100.nc')